In [1]:
import torch
import random
import numpy as np
from models import TFModel
from utils import create_folder, fix_random_seed, Config
from task_generate import gen_default_tokernizer, simpleReasoning
from train import make_scheduler, train_fresh_sample
import yaml
import json

with open("./config.yaml", "r") as file:
    config_args = yaml.safe_load(file)
config = Config(**config_args)
fix_random_seed(config.seed)

In [2]:
tokenizer = gen_default_tokernizer()
vocab_size = len(tokenizer["map"])
config.vocab_size = vocab_size

# try generating some example
task = simpleReasoning(tokenizer, max_variables=config.max_variables, max_parenthesis=config.max_parenthesis, max_seq_len=config.max_seq_len)
res, info = task.formatted_sample(num_steps=100)
print(json.dumps(task.get_task_details(), indent=4))

for ids in res[:5]:
    print(task.map_ids_to_str(ids))
    print("")

{
    "name": "simple_reasoning",
    "description": "simple arithmetic deduction steps",
    "max_variables": 12,
    "max_parenthesis": 7,
    "num_variables_samp_prob": null,
    "num_parentheses_samp_prob": null,
    "variable_samp_prob": null,
    "operator_samp_prob": null,
    "sample_size": 97
}
PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP(9-7-5-(1+1)-0)  >>>
(9-7-5-2-0)

PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP(0-0+5-3+0)-9-2+0  >>>
(0+5-3+0)-9-2+0

PPPPPPPPPPP((3+0+2)-(((2+6+(6-8))+1)))  >>>
((3+2)-(((2+6+(6-8))+1)))

PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP5-(1+8+0)-9  >>>
5-(9+0)-9

PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP(8+6+3)  >>>
(4+3)



In [3]:
model = TFModel(config).to(config.device)
config.print_output = True

In [4]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.lr,
    betas=(0.9, 0.98),
    eps=1e-9,
    weight_decay=config.wd if config.use_wd else 0,
)

scheduler = make_scheduler(optimizer, config)


In [5]:
model, training_info = train_fresh_sample(model, config, optimizer, scheduler)

  0%|          | 1/500 [00:00<07:17,  1.14it/s]

----> Epoch:     1, Test Loss: 2.727, Test Error: 0.889


  0%|          | 2/500 [00:01<05:41,  1.46it/s]

----> Epoch:     2, Test Loss: 2.351, Test Error: 0.907


  1%|          | 3/500 [00:01<04:52,  1.70it/s]

...

 96%|█████████▌| 481/500 [03:57<00:09,  2.03it/s]

----> Epoch:   481, Test Loss: 1.877, Test Error: 0.733


 98%|█████████▊| 491/500 [04:02<00:04,  1.97it/s]

----> Epoch:   491, Test Loss: 1.866, Test Error: 0.736


100%|██████████| 500/500 [04:07<00:00,  2.02it/s]
